<a href="https://colab.research.google.com/github/irenezi/solid-guide/blob/master/coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from pandas import read_csv, set_option
from numpy import set_printoptions
from matplotlib import pyplot
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import Normalizer, MinMaxScaler
from sklearn.feature_selection import VarianceThreshold, SelectKBest, chi2, RFE
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import ExtraTreesClassifier
from keras.layers import Dense, Input 
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l1

In [ ]:
def filter_constant_columns(df_data_set_x, variance=0):
  # This function filters out all the constant columns (variance = 0)
  
  constant_filter = VarianceThreshold(threshold=variance)
  constant_filter.fit(df_data_set_x)
  non_constant = constant_filter.get_support()

  columns_to_keep = (df_data_set_x.columns[constant_filter.get_support()])

  return df_data_set_x[columns_to_keep]

In [ ]:
def normalize(x_train_input, x_test_input):
  # Normalize train and test data
  # Normalize the training set and save the normalization parameters
  # Normalize the test set using the training normalization parameters

  scaler=Normalizer().fit(x_train_input)
  x_train_norm=scaler.transform(x_train_input)
  x_test_norm=scaler.transform(x_test_input)
  
  return x_train_norm, x_test_norm

In [ ]:
def rescale(x_train_input, x_test_input):
  # Rescale train and test data
  # Rescale the training set and save the normalization parameters
  # Rescale the test set using the training normalization parameters

  scaler=MinMaxScaler(feature_range=(0,1))
  scaler = scaler.fit(x_train_input)
  x_train_resc = scaler.transform(x_train_input)
  x_test_resc = scaler.transform(x_test_input)
  return x_train_resc, x_test_resc

In [ ]:
def compare_algorithms(x_train_input, y_train_output, x_test_input, y_test_output):

  # prints the validation accuracy (using K-Folf method on the train data set)
  # and the test accuracy by apllying a variety of models to the test data set  
  
  models=[]
  models.append(('LR', LogisticRegression(solver='liblinear'))) 
  models.append(('LDA', LinearDiscriminantAnalysis()))
  models.append(('KNN', KNeighborsClassifier()))
  #models.append(('CART', DecisionTreeClassifier()))
  #models.append(('NB', GaussianNB()))
  models.append(('SVC', SVC()))
  models.append(('LnSVC', LinearSVC()))

  print("Validation accuracy:")
  for name, model in models:
    kfold = KFold(n_splits=5, random_state=3, shuffle=True)
    cv_results = cross_val_score(model,x_train_input,y_train_output,cv=kfold,scoring="accuracy")
    print("%s: %.3f (%.3f)" % (name,cv_results.mean(),cv_results.std()))
  
  print("Test accuracy:")
  for name, model in models:
    model.fit(x_train_input, y_train_output)
    test_prediction = model.predict(x_test_input)
    test_acc = accuracy_score(y_test_output, test_prediction)
    print("%s: %.3f" % (name,test_acc))


In [ ]:
def kbest(x_train_input, y_train_output, x_test_input):
  # feature selection using K-best
  
  # apply SelectKBest class to extract top 10 best features
  bestfeatures = SelectKBest(score_func=chi2, k=10)
  fit = bestfeatures.fit(x_train_input, y_train_output)
  dfscores = pd.DataFrame(fit.scores_)
  df = pd.DataFrame(x_train_input)
  dfcolumns = pd.DataFrame(df.columns)

  # concat two dataframes for better visualization 
  featureScores = pd.concat([dfcolumns,dfscores],axis=1)
  # naming the dataframe columns
  featureScores.columns = ['Specs','Score']  

  # print 10 best features
  print(featureScores.nlargest(10,'Score'))  

  # find the columns of the 10 best features
  featureScores=featureScores.nlargest(10,'Score').values
  col=list(featureScores[:,0])

  # returned train data set
  features = fit.transform(x_train_input)

  # returned test data set
  df_test_features = pd.DataFrame(x_test_input)
  test_features = df_test_features[col]

  return features, test_features.values

In [ ]:
def rfe_elimination(model, x_train_input, y_train_output, x_test_input):
  # apply Recursive Feature Elimination(RFE) for feature selection

  rfe=RFE(model,10)
  fit=rfe.fit(x_train_input, y_train_output)
  #print("Num Features: %d" % fit.n_features_)
  #print("Selected Features: %s" % fit.support_)
  #print("Feature Ranking: %s" % fit.ranking_)

  dfscores = pd.DataFrame(fit.ranking_)
  df = pd.DataFrame(x_train_input)
  dfcolumns = pd.DataFrame(df.columns)

  # concat two dataframes for better visualization 
  featureScores = pd.concat([dfcolumns,dfscores],axis=1)
  # naming the dataframe columns
  featureScores.columns = ['Specs','Score']  
  print(featureScores.nsmallest(10,'Score')) 

  # find the columns of the 10 best features
  featureScores=(featureScores.nsmallest(10,'Score')).values
  col=list(featureScores[:,0])
  print(col)

  # returned train data set
  features=fit.transform(x_train_input)

  # returned test data set
  df_test_features = pd.DataFrame(x_test_input)
  test_features = df_test_features[col]

  return features, test_features.values


In [33]:
def extra_trees_classifier(x_train_input, y_train_output, x_test_input):
  # feature selection with extra tree classifier - Feature Importance

  model = ExtraTreesClassifier()
  tree=model.fit(x_train_input,y_train_output)
  #print(model.feature_importances_)

  dfscores = pd.DataFrame(tree.feature_importances_)
  df = pd.DataFrame(x_train_input)
  dfcolumns = pd.DataFrame(df.columns)
  
  # concat two dataframes for better visualization 
  featureScores = pd.concat([dfcolumns,dfscores],axis=1)
  # naming the dataframe columns
  featureScores.columns = ['Specs','Score']  
  featureScores=featureScores.nlargest(10,'Score')
  print(featureScores)

  # find the columns of the 10 best features
  featureScores=featureScores.values
  col=list(featureScores[:,0])

  # returned train data set
  features=df[col]

  # returned test data set
  df_test_features = pd.DataFrame(x_test_input)
  test_features = df_test_features[col]

  return features.values, test_features.values


In [ ]:
#no need to reshape!!!!
#reshape
#x_train_norm = x_train_norm.reshape((len(x_train_norm), np.prod(x_train_norm.shape[1:]))) 
#x_test_norm = x_test_norm.reshape((len(x_test_norm), np.prod(x_test_norm.shape[1:])))
#print(x_train_norm.shape)

(97044, 152)


In [ ]:
def autoencoder(x_train_input, x_test_input):
  # returns the encoder model of a simple autoencoder with 2 hidden layers

  input = Input(shape=(82,))
  hidden_1 = Dense(500, activation='relu')(input)
  hidden_2 = Dense(300, activation='relu')(hidden_1)
  code = Dense(20, activation='relu')(hidden_2)
  hidden_3 = Dense(300, activation='relu')(code)
  hidden_4 = Dense(500, activation='relu')(hidden_3) 
  output = Dense(82, activation='sigmoid')(hidden_4)

  # define autoencoder model
  autoencoder = Model(inputs=input, outputs=output)
  # compile model
  autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
  # fit model to reconstruct the input
  autoencoder.fit(x_train_input, x_train_input, batch_size=40, epochs=3, validation_data=(x_test_input, x_test_input))

  # define encoder model
  encoder1 = Model(inputs=input, outputs=code)
#  # save the encoder model to file
#  encoder.save("encoder1.h5")

  return encoder1

In [ ]:
def denoising_autoencoder(x_train_input, x_test_input):
  # returns the encoder model of a denoising autoencoder with 2 hidden layers

  # add noise
  noise_factor = 0.4
  x_train_noisy = x_train_input + noise_factor * np.random.normal(size=x_train_input.shape)
  x_test_noisy = x_test_input + noise_factor * np.random.normal(size=x_test_input.shape)
  x_train_noisy = np.clip(x_train_noisy, 0.0, 1.0)
  x_test_noisy = np.clip(x_test_noisy, 0.0, 1.0)

  input = Input(shape=(82,))
  hidden_1 = Dense(500, activation='relu')(input)
  hidden_2 = Dense(300, activation='relu')(hidden_1)
  code = Dense(20, activation='relu')(hidden_2)
  hidden_3 = Dense(300, activation='relu')(code)
  hidden_4 = Dense(500, activation='relu')(hidden_3) 
  output = Dense(82, activation='sigmoid')(hidden_4)

  #define noisy aytoencoder model
  autoencoder_noisy = Model(inputs=input, outputs=output)
  # compile the model
  autoencoder_noisy.compile(optimizer='adam', loss='binary_crossentropy')
  # fit model to reconstruct the input
  autoencoder_noisy.fit(x_train_noisy, x_train_noisy, batch_size=40, epochs=3, validation_data=(x_test_noisy, x_test_noisy))

  # define the encoder model
  encoder2 = Model(inputs=input, outputs=code)
  #  # save the encoder model to file
  #  encoder.save("encoder2.h5")

  return encoder2

In [ ]:
def sparse_autoencoder(x_train_input, x_test_input):
  #returns the encoder model of a sparse autoencoder 

  input_size = 82
  code_size = 20

  input = Input(shape=(input_size,))
  code = Dense(code_size,activity_regularizer=l1(10e-6), activation='relu')(input) 
  output = Dense(input_size, activation='sigmoid')(code)

  # define autoencoder model
  autoencoder_regularized = Model(inputs=input, outputs=output) 
  # compile the autoencoder model
  autoencoder_regularized.compile(optimizer='adam', loss='binary_crossentropy') 
  # fit autoencoder model to reconstruct the input
  autoencoder_regularized.fit(x_train_input, x_train_input, epochs=3, validation_data=(x_test_input, x_test_input))
  
  # define the encoder model
  encoder3 = Model(input, code)
  #  # save the encoder model to file
  #  encoder.save("encoder3.h5")

  return encoder3

In [ ]:
def encoder_transformation(encoder, x_train_input, x_test_input):
  # use the encoder models to transform the data

  # train_data transformation
  x_train_encoded = encoder.predict(x_train_input)
  # test data transformation
  x_test_encoded = encoder.predict(x_test_input)

  return x_train_encoded, x_test_encoded

In [ ]:
def mixed_data(train1, train2, test1, test2):
  # concatenate columns wise 2 numpy arrays

  train_array = np.concatenate((train1, train2), axis=1)
  test_array = np.concatenate((test1, test2), axis=1)

  return train_array, test_array

In [41]:
def compare_algorithms_2(x_train_input, y_train_output, x_test_input, y_test_output):

  # prints the validation accuracy (using K-Folf method on the train data set)
  # and the test accuracy by applying a selection of models from 
  # compare_algorithms method to the test data set  
  
  models=[]
  models.append(('LR', LogisticRegression(solver='liblinear'))) 
  models.append(('LDA', LinearDiscriminantAnalysis()))
  models.append(('LnSVC', LinearSVC()))

  print("Validation accuracy:")
  for name, model in models:
    kfold = KFold(n_splits=5, random_state=3, shuffle=True)
    cv_results = cross_val_score(model,x_train_input,y_train_output,cv=kfold,scoring="accuracy")
    print("%s: %.3f (%.3f)" % (name,cv_results.mean(),cv_results.std()))
  
  print("Test accuracy:")
  for name, model in models:
    model.fit(x_train_input, y_train_output)
    test_prediction = model.predict(x_test_input)
    test_acc = accuracy_score(y_test_output, test_prediction)
    print("%s: %.3f" % (name,test_acc))

In [ ]:
def create_nn_model(init="glorot_uniform", optimizer="rmsprop"):
  #returns a neural network nodel

  model=Sequential()
  model.add(Dense(150, input_dim=82, activation="relu", kernel_initializer=init))
  model.add(Dense(50, activation="relu", kernel_initializer=init))
  model.add(Dense(1, activation="softmax", kernel_initializer=init))

  model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

  return model

In [ ]:
# Grid search for NN

def grid_search(input, output):
  #numpy.set_printoptions(precision=3)

  # create model
  nn_model=KerasClassifier(build_fn=create_nn_model, verbose=0)

  #grid search to find epochs, batch_size, optimizer
  epochs_list=[5,10,30,50]
  batches_list=[5,10,15]
  optimizer_list=["adam", "rmsprop"]
  inits_list=["glorot_uniform","normal", "uniform"]

  param_grid_dictionary=dict(optimizer=optimizer_list, epochs=epochs_list, batch_size=batches_list, init=inits_list)
  type(param_grid_dictionary)
  grid=GridSearchCV(estimator=nn_model,param_grid=param_grid_dictionary)
  grid_result=grid.fit(input,output)

  # summarize results
  print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
  means = grid_result.cv_results_['mean_test_score']
  stds = grid_result.cv_results_['std_test_score']
  params = grid_result.cv_results_['params']
  for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


In [ ]:
def reduced_data(df_data, new_size=600):
  # creates a reduced dataset in order to minimize time needed for grid search

  number_of_rows = df_data.shape[0]
  random_indices = np.random.choice(number_of_rows, size=new_size, replace=False)
  random_rows = df_data.values[random_indices, :]

  X=random_rows[:,0:82]
  Y=random_rows[:,82]

  return X,Y

In [ ]:
# use a reduced (train) data set for grid search 
reduced_x, reduced_y = reduced_data(df_train_data_filtered)

In [ ]:
grid_search(X,Y)
"""Best: 0.471667 using {'batch_size': 5, 'epochs': 5, 'init': 'glorot_uniform', 'optimizer': 'adam'}"""

RUN

DEALING WITH THE DATA

In [ ]:
#load the data
filename1 ="train_imperson_without4n7_balanced_data.csv"
filename2 ="test_imperson_without4n7_balanced_data.csv"

df_train_data = read_csv(filename1)
df_test_data = read_csv(filename2)


# apply constant column filter

# not matched constant columns in the train and test data set

# merge them to one data set
df_full_data = df_train_data.append(df_test_data)

#apply the filter
df_full_data_filtered = filter_constant_columns(df_full_data)

# separate the data sets
df_train_data_filtered = df_full_data_filtered[0:97044]
df_test_data_filtered = df_full_data_filtered[97044:]


#separate the filtered x,y
x_train = df_train_data_filtered[df_train_data_filtered.columns[:-1]].values
y_train = df_train_data_filtered[df_train_data_filtered.columns[-1]].values
x_test = df_test_data_filtered[df_test_data_filtered.columns[:-1]].values
y_test = df_test_data_filtered[df_test_data_filtered.columns[-1]].values

In [ ]:
x_train_norm, x_test_norm = normalize(x_train, x_test)

In [ ]:
x_train_resc, x_test_resc = rescale(x_train, x_test)

FITTING MODELS TO COMPARE, USING: Raw data / Normalized data / Rescaled data

In [ ]:
# fitting models to raw data
compare_algorithms(x_train, y_train, x_test, y_test)

Validation accuracy:
LR: 0.992 (0.001)
LDA: 0.985 (0.002)
KNN: 0.999 (0.000)
SVC: 0.995 (0.001)
LnSVC: 0.995 (0.001)
Test accuracy:
LR: 0.868
LDA: 0.931
KNN: 0.534
SVC: 0.530
LnSVC: 0.652


In [ ]:
# fitting models to normalized data
compare_algorithms(x_train_norm, y_train, x_test_norm, y_test)

Validation accuracy:
LR: 0.988 (0.001)
LDA: 0.985 (0.002)
KNN: 0.999 (0.000)
SVC: 0.995 (0.001)
LnSVC: 0.992 (0.001)
Test accuracy:
LR: 0.965
LDA: 0.614
KNN: 0.532
SVC: 0.531
LnSVC: 0.525


In [ ]:
# fitting models to rescaled data
compare_algorithms(x_train_resc, y_train, x_test_resc, y_test)

Validation accuracy:
LR: 0.995 (0.001)
LDA: 0.985 (0.002)
KNN: 1.000 (0.000)
SVC: 0.997 (0.001)
LnSVC: 0.997 (0.001)
Test accuracy:
LR: 0.506
LDA: 0.931
KNN: 0.536
SVC: 0.499
LnSVC: 0.509


FEATURE SELECTION (TOP 10 FEATURES) USING: K-BEST / RFE / EXTRA TREES

In [ ]:
# K-Best method for raw data
kbest_train_features, kbest_test_features = kbest(x_train, y_train, x_test)

    Specs         Score
26     26  32124.682195
16     16  28936.000138
22     22  28321.490224
13     13  24346.021809
81     81  13311.733744
15     15  12317.666392
2       2  12076.290588
3       3  12076.290588
23     23   7911.296191
36     36   7720.479234


In [ ]:
# K-Best method for normalized data
kbest_train_features_norm, kbest_test_features_norm = kbest(x_train_norm, y_train, x_test_norm)

    Specs        Score
26     26  7185.552254
22     22  6458.853639
16     16  6265.412604
13     13  5162.369647
81     81  2648.775398
15     15  2569.185754
2       2  2378.606418
3       3  2378.606418
23     23  2091.641720
28     28  1758.070006


In [ ]:
# K-Best method for rescaled data
kbest_train_features_resc, kbest_test_features_resc = kbest(x_train_resc, y_train, x_test_resc)

    Specs         Score
26     26  32124.682195
16     16  28936.000138
22     22  28321.490224
13     13  24346.021809
81     81  13311.733744
15     15  12317.666392
2       2  12076.290588
3       3  12076.290588
23     23   7911.296191
36     36   7720.479234


In [ ]:
# Recursive Feature Elimination method for raw data
rfe_train_features, rfe_test_features = rfe_elimination(LogisticRegression(solver='liblinear'), x_train, y_train, x_test)

    Specs  Score
3       3      1
13     13      1
22     22      1
25     25      1
36     36      1
44     44      1
46     46      1
50     50      1
55     55      1
74     74      1
[3, 13, 22, 25, 36, 44, 46, 50, 55, 74]


In [ ]:
# Recursive Feature Elimination method for normalized data
rfe_train_features_norm, rfe_test_features_norm = rfe_elimination(LogisticRegression(solver='liblinear'), x_train_norm, y_train, x_test_norm)

    Specs  Score
13     13      1
15     15      1
18     18      1
22     22      1
26     26      1
36     36      1
44     44      1
46     46      1
69     69      1
73     73      1
[13, 15, 18, 22, 26, 36, 44, 46, 69, 73]


In [ ]:
# Recursive Feature Elimination method for rescaled data
rfe_train_features_resc, rfe_test_features_resc = rfe_elimination(LogisticRegression(solver='liblinear'), x_train_resc, y_train, x_test_resc)

    Specs  Score
13     13      1
20     20      1
23     23      1
25     25      1
36     36      1
50     50      1
51     51      1
55     55      1
62     62      1
74     74      1
[13, 20, 23, 25, 36, 50, 51, 55, 62, 74]


In [ ]:
# Extra Trees Classifier method for raw data
extree_train_features, extree_test_features = extra_trees_classifier(x_train, y_train, x_test)

    Specs     Score
22     22  0.106122
26     26  0.100525
15     15  0.078155
13     13  0.074158
23     23  0.067266
16     16  0.059308
28     28  0.058513
2       2  0.053410
36     36  0.043418
3       3  0.038511


In [34]:
# Extra Trees Classifier method for normalized data
extree_train_features_norm, extree_test_features_norm = extra_trees_classifier(x_train_norm, y_train, x_test_norm)

    Specs     Score
15     15  0.103090
16     16  0.072870
23     23  0.070111
26     26  0.066004
22     22  0.056709
13     13  0.055101
28     28  0.048259
11     11  0.047881
36     36  0.035849
2       2  0.029688


In [ ]:
# Extra Trees Classifier method for rescaled data
extree_train_features_resc, extree_test_features_resc = extra_trees_classifier(x_train_resc, y_train, x_test_resc)

    Specs     Score
22     22  0.107082
26     26  0.098998
15     15  0.091695
16     16  0.082208
28     28  0.079847
23     23  0.067989
36     36  0.060018
13     13  0.055606
81     81  0.036854
11     11  0.034167


MODEL EVALUATION WITH SELECTED FEATURES

In [ ]:
# raw data

In [ ]:
# compare models using top 10 features from K-Best for raw data
compare_algorithms(kbest_train_features, y_train, kbest_test_features, y_test)

Validation accuracy:
LR: 0.961 (0.001)
LDA: 0.928 (0.001)
KNN: 0.998 (0.000)
SVC: 0.992 (0.001)
LnSVC: 0.960 (0.001)
Test accuracy:
LR: 0.500
LDA: 0.277
KNN: 0.500
SVC: 0.500
LnSVC: 0.500


In [ ]:
# compare models using top 10 features from RFE for raw data
compare_algorithms(rfe_train_features, y_train, rfe_test_features, y_test)

Validation accuracy:
LR: 0.985 (0.001)
LDA: 0.969 (0.002)
KNN: 0.999 (0.000)
SVC: 0.997 (0.000)
LnSVC: 0.993 (0.001)
Test accuracy:
LR: 0.861
LDA: 0.820
KNN: 0.531
SVC: 0.521
LnSVC: 0.866


In [ ]:
# compare models using top 10 features from Extra Trees for raw data
compare_algorithms(extree_train_features, y_train, extree_test_features, y_test)

Validation accuracy:
LR: 0.961 (0.001)
LDA: 0.932 (0.002)
KNN: 0.998 (0.000)
SVC: 0.992 (0.001)
LnSVC: 0.961 (0.001)
Test accuracy:
LR: 0.975
LDA: 0.910
KNN: 0.533
SVC: 0.527
LnSVC: 0.975


In [ ]:
# normalized data

In [ ]:
# compare models using top 10 features from K-Best for normalized data
compare_algorithms(kbest_train_features_norm, y_train, kbest_test_features_norm, y_test)

Validation accuracy:
LR: 0.929 (0.002)
LDA: 0.928 (0.002)
KNN: 0.997 (0.000)
SVC: 0.981 (0.002)
LnSVC: 0.941 (0.002)
Test accuracy:
LR: 0.500
LDA: 0.675
KNN: 0.500
SVC: 0.500
LnSVC: 0.499


In [ ]:
# compare models using top 10 features from RFE for normalized data
compare_algorithms(rfe_train_features_norm, y_train, rfe_test_features_norm, y_test)

Validation accuracy:
LR: 0.987 (0.001)
LDA: 0.974 (0.002)
KNN: 0.998 (0.000)
SVC: 0.991 (0.001)
LnSVC: 0.988 (0.002)
Test accuracy:
LR: 0.520
LDA: 0.958
KNN: 0.532
SVC: 0.527
LnSVC: 0.519


In [ ]:
# compare models using top 10 features from Extra Trees for normalized data
compare_algorithms(extree_train_features_norm, y_train, extree_test_features_norm, y_test)

Validation accuracy:
LR: 0.931 (0.002)
LDA: 0.929 (0.002)
KNN: 0.999 (0.000)
SVC: 0.993 (0.000)
LnSVC: 0.950 (0.001)
Test accuracy:
LR: 0.917
LDA: 0.887
KNN: 0.528
SVC: 0.521
LnSVC: 0.966


In [ ]:
# rescaled data

In [ ]:
# compare models using top 10 features from K-Best for rescaled data
compare_algorithms(kbest_train_features_resc, y_train, kbest_test_features_resc, y_test)

Validation accuracy:
LR: 0.961 (0.001)
LDA: 0.928 (0.001)
KNN: 0.998 (0.000)
SVC: 0.992 (0.001)
LnSVC: 0.960 (0.001)
Test accuracy:
LR: 0.500
LDA: 0.277
KNN: 0.500
SVC: 0.500
LnSVC: 0.500


In [ ]:
# compare models using top 10 features from RFE for rescaled data
compare_algorithms(rfe_train_features_resc, y_train, rfe_test_features_resc, y_test)

Validation accuracy:
LR: 0.992 (0.001)
LDA: 0.980 (0.002)
KNN: 0.999 (0.000)
SVC: 0.997 (0.001)
LnSVC: 0.995 (0.001)
Test accuracy:
LR: 0.813
LDA: 0.783
KNN: 0.535
SVC: 0.494
LnSVC: 0.816


In [ ]:
# compare models using top 10 features from Extra Trees for rescaled data
compare_algorithms(extree_train_features_resc, y_train, extree_test_features_resc, y_test)

Validation accuracy:
LR: 0.969 (0.002)
LDA: 0.938 (0.002)
KNN: 0.997 (0.000)
SVC: 0.988 (0.000)
LnSVC: 0.971 (0.002)
Test accuracy:
LR: 0.959
LDA: 0.916
KNN: 0.521
SVC: 0.502
LnSVC: 0.958


USE OF AUTOENCODERS FOR FEATURE EXTRACTION: SIMPLE / DENOISING / SPARSE

---




In [24]:
# creating the encoder model of the simple autoencoder
# for the raw, normalized and rescaled data

encoder1_raw = autoencoder(x_train, x_test)
encoder1_norm = autoencoder(x_train_norm, x_test_norm)
encoder1_resc = autoencoder(x_train_resc, x_test_resc)


Epoch 1/3
2427/2427 [==============================] - 20s 8ms/step - loss: 0.0857 - val_loss: 0.3677
Epoch 2/3
2427/2427 [==============================] - 19s 8ms/step - loss: 0.0595 - val_loss: 0.3975
Epoch 3/3
2427/2427 [==============================] - 19s 8ms/step - loss: 0.0592 - val_loss: 0.4044
Epoch 1/3
2427/2427 [==============================] - 20s 8ms/step - loss: 0.1678 - val_loss: 0.1705
Epoch 2/3
2427/2427 [==============================] - 19s 8ms/step - loss: 0.1470 - val_loss: 0.1682
Epoch 3/3
2427/2427 [==============================] - 19s 8ms/step - loss: 0.1469 - val_loss: 0.1691
Epoch 1/3
2427/2427 [==============================] - 21s 8ms/step - loss: 0.0788 - val_loss: -2911187.2500
Epoch 2/3
2427/2427 [==============================] - 19s 8ms/step - loss: 0.0507 - val_loss: -1437030.0000
Epoch 3/3
2427/2427 [==============================] - 19s 8ms/step - loss: 0.0503 - val_loss: -2511608.0000


In [25]:
# creating the encoder model of the denoising autoencoder
# for the raw, normalized and rescaled data

encoder2_raw = denoising_autoencoder(x_train, x_test)
encoder2_norm = denoising_autoencoder(x_train_norm, x_test_norm)
encoder2_resc = denoising_autoencoder(x_train_resc, x_test_resc)

Epoch 1/3
2427/2427 [==============================] - 20s 8ms/step - loss: 0.4386 - val_loss: 0.4407
Epoch 2/3
2427/2427 [==============================] - 19s 8ms/step - loss: 0.4140 - val_loss: 0.4342
Epoch 3/3
2427/2427 [==============================] - 19s 8ms/step - loss: 0.4103 - val_loss: 0.4327
Epoch 1/3
2427/2427 [==============================] - 20s 8ms/step - loss: 0.4674 - val_loss: 0.4508
Epoch 2/3
2427/2427 [==============================] - 19s 8ms/step - loss: 0.4470 - val_loss: 0.4434
Epoch 3/3
2427/2427 [==============================] - 19s 8ms/step - loss: 0.4412 - val_loss: 0.4391
Epoch 1/3
2427/2427 [==============================] - 20s 8ms/step - loss: 0.4408 - val_loss: 0.4193
Epoch 2/3
2427/2427 [==============================] - 19s 8ms/step - loss: 0.4157 - val_loss: 0.4162
Epoch 3/3
2427/2427 [==============================] - 20s 8ms/step - loss: 0.4129 - val_loss: 0.4135


In [26]:
# creating the encoder model of sparse autoencoder
# for the raw, normalized and rescaled data
encoder3_raw = sparse_autoencoder(x_train, x_test)
encoder3_norm = sparse_autoencoder(x_train_norm, x_test_norm)
encoder3_resc = sparse_autoencoder(x_train_resc, x_test_resc) 

Epoch 1/3
3033/3033 [==============================] - 6s 2ms/step - loss: 0.1608 - val_loss: 0.3386
Epoch 2/3
3033/3033 [==============================] - 5s 2ms/step - loss: 0.0623 - val_loss: 0.3589
Epoch 3/3
3033/3033 [==============================] - 5s 2ms/step - loss: 0.0605 - val_loss: 0.3517
Epoch 1/3
3033/3033 [==============================] - 6s 2ms/step - loss: 0.2197 - val_loss: 0.1675
Epoch 2/3
3033/3033 [==============================] - 5s 2ms/step - loss: 0.1494 - val_loss: 0.1685
Epoch 3/3
3033/3033 [==============================] - 5s 2ms/step - loss: 0.1479 - val_loss: 0.1690
Epoch 1/3
3033/3033 [==============================] - 5s 2ms/step - loss: 0.1403 - val_loss: -3136183.7500
Epoch 2/3
3033/3033 [==============================] - 5s 2ms/step - loss: 0.0536 - val_loss: -5242994.5000
Epoch 3/3
3033/3033 [==============================] - 5s 2ms/step - loss: 0.0518 - val_loss: -6170987.0000


In [27]:
# data transformation - encoder1
x_train_enc_1, x_test_enc_1 = encoder_transformation(encoder1_raw, x_train, x_test)
x_train_norm_enc_1, x_test_norm_enc_1 = encoder_transformation(encoder1_norm, x_train_norm, x_test_norm) 
x_train_resc_enc_1, x_test_resc_enc_1 = encoder_transformation(encoder1_resc, x_train_resc, x_test_resc)


In [28]:
# data transformation - encoder2
x_train_enc_2, x_test_enc_2 = encoder_transformation(encoder2_raw, x_train, x_test)
x_train_norm_enc_2, x_test_norm_enc_2 = encoder_transformation(encoder2_norm, x_train_norm, x_test_norm) 
x_train_resc_enc_2, x_test_resc_enc_2 = encoder_transformation(encoder2_resc, x_train_resc, x_test_resc)

In [29]:
# data transformation - encoder3
x_train_enc_3, x_test_enc_3 = encoder_transformation(encoder3_raw, x_train, x_test)
x_train_norm_enc_3, x_test_norm_enc_3 = encoder_transformation(encoder3_norm, x_train_norm, x_test_norm) 
x_train_resc_enc_3, x_test_resc_enc_3 = encoder_transformation(encoder3_resc, x_train_resc, x_test_resc)

FITTING MODELS TO EXTRACTED DATA

In [ ]:
# encoder1 - raw data
compare_algorithms(x_train_enc_1, y_train, x_test_enc_1, y_test)

Validation accuracy:
LR: 0.973 (0.002)
LDA: 0.950 (0.002)
KNN: 0.999 (0.000)
SVC: 0.997 (0.000)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

LnSVC: 0.974 (0.001)
Test accuracy:
LR: 0.569
LDA: 0.943
KNN: 0.534
SVC: 0.534
LnSVC: 0.566


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# encoder1 - normalized data
compare_algorithms(x_train_norm_enc_1, y_train, x_test_norm_enc_1, y_test)

Validation accuracy:
LR: 0.985 (0.002)
LDA: 0.967 (0.001)
KNN: 0.999 (0.000)
SVC: 0.995 (0.001)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

LnSVC: 0.985 (0.002)
Test accuracy:
LR: 0.868
LDA: 0.924
KNN: 0.518
SVC: 0.500
LnSVC: 0.874


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# encoder1 - rescaled data
compare_algorithms(x_train_resc_enc_1, y_train, x_test_resc_enc_1, y_test)

Validation accuracy:
LR: 0.994 (0.001)
LDA: 0.985 (0.001)
KNN: 0.999 (0.000)
SVC: 0.999 (0.000)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

LnSVC: 0.994 (0.001)
Test accuracy:
LR: 0.609
LDA: 0.810
KNN: 0.500
SVC: 0.500
LnSVC: 0.534


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# encoder2 - raw data
compare_algorithms(x_train_enc_2, y_train, x_test_enc_2, y_test)

Validation accuracy:
LR: 0.991 (0.001)
LDA: 0.980 (0.002)
KNN: 0.999 (0.000)
SVC: 0.995 (0.000)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

LnSVC: 0.993 (0.001)
Test accuracy:
LR: 0.552
LDA: 0.502
KNN: 0.531
SVC: 0.525
LnSVC: 0.532


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# encoder2 - normalized data
compare_algorithms(x_train_norm_enc_2, y_train, x_test_norm_enc_2, y_test)

Validation accuracy:
LR: 0.985 (0.001)
LDA: 0.953 (0.001)
KNN: 0.999 (0.000)
SVC: 0.980 (0.001)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

LnSVC: 0.990 (0.001)
Test accuracy:
LR: 0.945
LDA: 0.500
KNN: 0.740
SVC: 0.951
LnSVC: 0.929


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# encoder2 - rescaled data data
compare_algorithms(x_train_resc_enc_2, y_train, x_test_resc_enc_2, y_test)

Validation accuracy:
LR: 0.995 (0.001)
LDA: 0.978 (0.002)
KNN: 0.999 (0.000)
SVC: 0.998 (0.000)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

LnSVC: 0.995 (0.001)
Test accuracy:
LR: 0.508
LDA: 0.504
KNN: 0.500
SVC: 0.500
LnSVC: 0.506


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# encoder3 - raw data
compare_algorithms(x_train_enc_3, y_train, x_test_enc_3, y_test)

Validation accuracy:
LR: 0.992 (0.001)
LDA: 0.977 (0.002)
KNN: 0.999 (0.000)
SVC: 0.996 (0.001)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

LnSVC: 0.992 (0.001)
Test accuracy:
LR: 0.867
LDA: 0.824
KNN: 0.508
SVC: 0.598
LnSVC: 0.512


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# encoder3 - normalized data
compare_algorithms(x_train_norm_enc_3, y_train, x_test_norm_enc_3, y_test)

Validation accuracy:
LR: 0.988 (0.002)
LDA: 0.974 (0.002)
KNN: 0.999 (0.000)
SVC: 0.992 (0.001)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LnSVC: 0.990 (0.001)
Test accuracy:
LR: 0.896
LDA: 0.850
KNN: 0.531
SVC: 0.524
LnSVC: 0.866


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# encoder3 - rescaled data
compare_algorithms(x_train_resc_enc_3, y_train, x_test_resc_enc_3, y_test)

Validation accuracy:
LR: 0.991 (0.001)
LDA: 0.984 (0.001)
KNN: 0.999 (0.000)
SVC: 0.998 (0.000)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

LnSVC: 0.993 (0.000)
Test accuracy:
LR: 0.953
LDA: 0.803
KNN: 0.500
SVC: 0.581
LnSVC: 0.474


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


COMBINE ORIGINAL WITH EXTRACTED FEATURES (NORMALIZED DATA)



In [ ]:
# FULL SET OF ORIGINAL FEATURES + EXTRACTED FEATURES
# 82 + 20 = 102 FEATURES 

In [31]:
# simple encoder
x_train_full_enc_1, x_test_full_enc_1 = mixed_data(x_train_norm, x_train_norm_enc_1, x_test_norm, x_test_norm_enc_1)
# denoising encoder
x_train_full_enc_2, x_test_full_enc_2 = mixed_data(x_train_norm, x_train_norm_enc_2, x_test_norm, x_test_norm_enc_2)
# sparse encoder
x_train_full_enc_3, x_test_full_enc_3 = mixed_data(x_train_norm, x_train_norm_enc_3, x_test_norm, x_test_norm_enc_3)

In [ ]:
# SELECTED SET OF ORIGINAL FEATURES (EXTRA TREES CLASSIFIER) + EXTRACTED FEATURES
# 10 + 20 = 30 FEATURES

In [37]:
# simple encoder
x_train_sel_enc_1, x_test_sel_enc_1 = mixed_data(extree_train_features_norm, x_train_norm_enc_1, extree_test_features_norm, x_test_norm_enc_1)
# denoising encoder
x_train_sel_enc_2, x_test_sel_enc_2 = mixed_data(extree_train_features_norm, x_train_norm_enc_2, extree_test_features_norm, x_test_norm_enc_2)
# sparse encoder
x_train_sel_enc_3, x_test_sel_enc_3 = mixed_data(extree_train_features_norm, x_train_norm_enc_3, extree_test_features_norm, x_test_norm_enc_3)

COMBINE ORIGINAL WITH EXTRACTED FEATURES (NORMALIZED DATA) - MODEL EVALUATION (LR, LDA, LnSVC)

In [ ]:
# FULL SET OF ORIGINAL FEATURES + EXTRACTED FEATURES
# 82 + 20 = 102 FEATURES 

In [42]:
# simple encoder
compare_algorithms_2(x_train_full_enc_1, y_train, x_test_full_enc_1, y_test)

Validation accuracy:
LR: 0.994 (0.001)
LDA: 0.986 (0.002)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

LnSVC: 0.996 (0.001)
Test accuracy:
LR: 0.524
LDA: 0.509
LnSVC: 0.529


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [43]:
# denoising encoder
compare_algorithms_2(x_train_full_enc_2, y_train, x_test_full_enc_2, y_test)

Validation accuracy:
LR: 0.989 (0.001)
LDA: 0.990 (0.001)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

LnSVC: 0.987 (0.012)
Test accuracy:
LR: 0.964
LDA: 0.509
LnSVC: 0.847


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [44]:
# sparse encoder
compare_algorithms_2(x_train_full_enc_3, y_train, x_test_full_enc_3, y_test)

Validation accuracy:
LR: 0.992 (0.001)
LDA: 0.986 (0.002)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LnSVC: 0.993 (0.001)
Test accuracy:
LR: 0.902
LDA: 0.936
LnSVC: 0.723


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# SELECTED SET OF ORIGINAL FEATURES (EXTRA TREES CLASSIFIER) + EXTRACTED FEATURES
# 10 + 20 = 30 FEATURES

In [45]:
# simple encoder
compare_algorithms_2(x_train_sel_enc_1, y_train, x_test_sel_enc_1, y_test)

Validation accuracy:
LR: 0.994 (0.001)
LDA: 0.982 (0.002)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

LnSVC: 0.994 (0.001)
Test accuracy:
LR: 0.524
LDA: 0.923
LnSVC: 0.525


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [46]:
# denoising encoder
compare_algorithms_2(x_train_sel_enc_2, y_train, x_test_sel_enc_2, y_test)

Validation accuracy:
LR: 0.982 (0.001)
LDA: 0.982 (0.002)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

LnSVC: 0.988 (0.001)
Test accuracy:
LR: 0.908
LDA: 0.500
LnSVC: 0.574


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [47]:
# sparse encoder
compare_algorithms_2(x_train_sel_enc_3, y_train, x_test_sel_enc_3, y_test)

Validation accuracy:
LR: 0.991 (0.001)
LDA: 0.982 (0.002)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LnSVC: 0.993 (0.001)
Test accuracy:
LR: 0.848
LDA: 0.867
LnSVC: 0.526


FEATURE SELECTION (EXTRA TREES CLASSIFIER) ON COMBINATION OF ORIGINAL WITH EXTRACTED FEATURES (NORMALIZED DATA)


In [ ]:
# SELECTED 10 FEATURES FROM THE FULL SET OF ORIGINAL FEATURES + EXTRACTED FEATURES

In [48]:
# simple encoder
sel_x_train_full_enc_1, sel_x_test_full_enc_1 = extra_trees_classifier(x_train_full_enc_1, y_train, x_test_full_enc_1)
# denoising encoder
sel_x_train_full_enc_2, sel_x_test_full_enc_2 = extra_trees_classifier(x_train_full_enc_2, y_train, x_test_full_enc_2)
#sparse encoder
sel_x_train_full_enc_3, sel_x_test_full_enc_3 = extra_trees_classifier(x_train_full_enc_3, y_train, x_test_full_enc_3)

    Specs     Score
97     97  0.088245
15     15  0.080704
22     22  0.080538
26     26  0.075869
13     13  0.040988
91     91  0.039032
36     36  0.035981
23     23  0.035032
16     16  0.031639
21     21  0.031536
     Specs     Score
93      93  0.089084
100    100  0.084938
26      26  0.061713
23      23  0.059794
16      16  0.046773
22      22  0.041572
15      15  0.038765
13      13  0.031708
83      83  0.030450
28      28  0.030437
     Specs     Score
95      95  0.091458
22      22  0.068341
26      26  0.053628
99      99  0.053448
28      28  0.052526
13      13  0.041536
101    101  0.037126
83      83  0.036190
16      16  0.034397
23      23  0.034137


In [ ]:
# SELECTED 10 FEATURES FROM THE SELECTED SET OF ORIGINAL FEATURES + EXTRACTED FEATURES

In [49]:
# simple encoder
sel_x_train_sel_enc_1, sel_x_test_sel_enc_1 = extra_trees_classifier(x_train_sel_enc_1, y_train, x_test_sel_enc_1)
# denoising encoder
sel_x_train_sel_enc_2, sel_x_test_sel_enc_2 = extra_trees_classifier(x_train_sel_enc_2, y_train, x_test_sel_enc_2)
#sparse encoder
sel_x_train_sel_enc_3, sel_x_test_sel_enc_3 = extra_trees_classifier(x_train_sel_enc_3, y_train, x_test_sel_enc_3)

    Specs     Score
4       4  0.123063
3       3  0.122729
25     25  0.111877
0       0  0.067649
1       1  0.061625
2       2  0.056047
23     23  0.049463
5       5  0.048328
8       8  0.040654
10     10  0.040294
    Specs     Score
28     28  0.121598
4       4  0.099716
21     21  0.092191
1       1  0.079977
3       3  0.057535
0       0  0.055431
19     19  0.054522
6       6  0.046416
5       5  0.043341
11     11  0.043006
    Specs     Score
23     23  0.116627
19     19  0.079915
3       3  0.070149
27     27  0.067198
4       4  0.066553
6       6  0.057995
0       0  0.054221
5       5  0.047613
29     29  0.044280
11     11  0.038585


FEATURE SELECTION (EXTRA TREES CLASSIFIER) ON COMBINATION OF ORIGINAL WITH EXTRACTED FEATURES (NORMALIZED DATA) - MODEL EVALUATION (LR, LDA, LnSVC)

In [ ]:
# SELECTED 10 FEATURES FROM THE FULL SET OF ORIGINAL FEATURES + EXTRACTED FEATURES

In [50]:
# simple encoder
compare_algorithms_2(sel_x_train_full_enc_1, y_train, sel_x_test_full_enc_1, y_test)

Validation accuracy:
LR: 0.973 (0.002)
LDA: 0.974 (0.002)
LnSVC: 0.977 (0.001)
Test accuracy:
LR: 0.602
LDA: 0.940
LnSVC: 0.517


In [51]:
# denoising encoder
compare_algorithms_2(sel_x_train_full_enc_2, y_train, sel_x_test_full_enc_2, y_test)

Validation accuracy:
LR: 0.957 (0.002)
LDA: 0.935 (0.002)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

LnSVC: 0.967 (0.002)
Test accuracy:
LR: 0.859
LDA: 0.847
LnSVC: 0.939


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [52]:
# sparse encoder
compare_algorithms_2(sel_x_train_full_enc_3, y_train, sel_x_test_full_enc_3, y_test)

Validation accuracy:
LR: 0.961 (0.002)
LDA: 0.971 (0.002)
LnSVC: 0.978 (0.001)
Test accuracy:
LR: 0.916
LDA: 0.898
LnSVC: 0.872


In [ ]:
# SELECTED 10 FEATURES FROM THE SELECTED SET OF ORIGINAL FEATURES + EXTRACTED FEATURES

In [54]:
# simple encoder
compare_algorithms_2(sel_x_train_sel_enc_1, y_train, sel_x_test_sel_enc_1, y_test)

Validation accuracy:
LR: 0.984 (0.001)
LDA: 0.969 (0.001)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LnSVC: 0.986 (0.001)
Test accuracy:
LR: 0.512
LDA: 0.925
LnSVC: 0.519


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [55]:
# denoising encoder
compare_algorithms_2(sel_x_train_sel_enc_2, y_train, sel_x_test_sel_enc_2, y_test)

Validation accuracy:
LR: 0.957 (0.002)
LDA: 0.934 (0.002)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

LnSVC: 0.965 (0.002)
Test accuracy:
LR: 0.859
LDA: 0.814
LnSVC: 0.887


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [56]:
# sparse encoder
compare_algorithms_2(sel_x_train_sel_enc_3, y_train, sel_x_test_sel_enc_2, y_test)

Validation accuracy:
LR: 0.979 (0.002)
LDA: 0.971 (0.002)
LnSVC: 0.982 (0.001)
Test accuracy:
LR: 0.668
LDA: 0.500
LnSVC: 0.500
